<H3>VI</h3>

In [21]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from langchain import hub
from langchain.agents import AgentExecutor,create_openai_tools_agent,create_react_agent,create_tool_calling_agent
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.prompts import ChatPromptTemplate
from pprint import pprint

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmOpenAI=ChatOpenAI(model="gpt-4o-mini")

In [4]:
wikipediaAPIWrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=100)
tools=[WikipediaQueryRun(api_wrapper=wikipediaAPIWrapper)]

In [5]:
prompt=hub.pull(owner_repo_commit="hwchase17/react")

In [6]:
tools[0].name

'wikipedia'

<h3>Will Give Error</h3>

In [7]:
agent=create_react_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [8]:
agentExecutor.invoke(input={"input":"What is Leo DiCaprio's middle name?\n\nAction: wikipedia"})



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I need to find information about Leo DiCaprio's middle name. I'll look it up on Wikipedia.

Action Input: "Leonardo DiCaprio"
`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [9]:
print(prompt.to_json()['kwargs']['template'])

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [10]:
# We can change the Prompt and agent for it to work
prompt=hub.pull("hwchase17/openai-tools-agent")
agent=create_openai_tools_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [11]:
agentExecutor.invoke(input={"input":"What is Leo DiCaprio's middle name?\n\nThought\n\nAction: Wikipedia"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Leonardo DiCaprio'}`


Page: Leonardo DiCaprio
Summary: Leonardo Wilhelm DiCaprio ( ; Italian: [diˈkaːprjo]; born November Leonardo DiCaprio's middle name is Wilhelm.

> Finished chain.


{'input': "What is Leo DiCaprio's middle name?\n\nThought\n\nAction: Wikipedia",
 'output': "Leonardo DiCaprio's middle name is Wilhelm."}

In [12]:
prompt.to_json()['kwargs']['messages']

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [13]:
# Else we can Parse it
prompt=hub.pull(owner_repo_commit="hwchase17/react")
agent=create_react_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True,handle_parsing_errors=True)

In [14]:
agentExecutor.invoke(input={"input":"What is Leo DiCaprio's middle name?\n\nAction: Wikipedia\n\nThought"})



> Entering new AgentExecutor chain...
I need to find out what Leo DiCaprio's middle name is by looking it up on Wikipedia.

Action: wikipedia  
Action Input: "Leonardo DiCaprio"  
Page: Leonardo DiCaprio
Summary: Leonardo Wilhelm DiCaprio ( ; Italian: [diˈkaːprjo]; born November I found Leo DiCaprio's full name, which includes his middle name.

Action: wikipedia  
Action Input: "Leonardo Wilhelm DiCaprio"  
Page: Leonardo DiCaprio
Summary: Leonardo Wilhelm DiCaprio ( ; Italian: [diˈkaːprjo]; born November I have the information I need about Leo DiCaprio's middle name, which is Wilhelm. 

Final Answer: Leo DiCaprio's middle name is Wilhelm.

> Finished chain.


{'input': "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia\n\nThought",
 'output': "Leo DiCaprio's middle name is Wilhelm."}

<h3>Apply Custom Error Messages</h3>

In [15]:
parsingError="Check your output and make sure it conforms, use the Action/Action Input syntax"
prompt=hub.pull(owner_repo_commit="hwchase17/react")
agent=create_react_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True,handle_parsing_errors=parsingError)

In [16]:
agentExecutor.invoke(input={"input":"What is Leo DiCaprio's middle name?\n\nAction: Wikipedia\n\nThought"})



> Entering new AgentExecutor chain...
Action: wikipedia  
Action Input: "Leonardo DiCaprio middle name"  Page: Leonardo DiCaprio
Summary: Leonardo Wilhelm DiCaprio ( ; Italian: [diˈkaːprjo]; born November I now know that Leonardo DiCaprio's middle name is Wilhelm.  
Final Answer: Wilhelm

> Finished chain.


{'input': "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia\n\nThought",
 'output': 'Wilhelm'}

<h3> Apply Custom Error Function</h3>

In [17]:
def _handleError(error):
    return str(error)
prompt=hub.pull(owner_repo_commit="hwchase17/react")
agent=create_react_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True,handle_parsing_errors=_handleError)

In [18]:
agentExecutor.invoke(input={"input":"What is Leo DiCaprio's middle name?\n\nAction: Wikipedia\n\nThought"})



> Entering new AgentExecutor chain...
I'm going to look up Leo DiCaprio's middle name on Wikipedia to find accurate information.

Action: wikipedia
Action Input: "Leonardo DiCaprio middle name"
Page: Leonardo DiCaprio
Summary: Leonardo Wilhelm DiCaprio ( ; Italian: [diˈkaːprjo]; born November I have found that Leonardo DiCaprio's middle name is Wilhelm.

Final Answer: Wilhelm

> Finished chain.


{'input': "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia\n\nThought",
 'output': 'Wilhelm'}

<h3>Access Intermediate Steps</h3>

In [19]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        ("system","You are a helpful assistant."),
        ("placeholder","{chat_history}"),
        ("human","{input}"),
        ("placeholder","{agent_scratchpad}")
    ]
)
agent=create_openai_tools_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True,return_intermediate_steps=True,handle_parsing_errors=True)

In [23]:
result=agentExecutor.invoke(input={"input":"What is Leo DiCaprio's middle name?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Leonardo DiCaprio'}`


Page: Leonardo DiCaprio
Summary: Leonardo Wilhelm DiCaprio ( ; Italian: [diˈkaːprjo]; born November Leonardo DiCaprio's middle name is Wilhelm.

> Finished chain.


In [24]:
pprint(result)

{'input': "What is Leo DiCaprio's middle name?",
 'intermediate_steps': [(ToolAgentAction(tool='wikipedia', tool_input={'query': 'Leonardo DiCaprio'}, log="\nInvoking: `wikipedia` with `{'query': 'Leonardo DiCaprio'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_R0lWhR4lX7CfJZ8VgzavO47N', 'function': {'arguments': '{"query":"Leonardo DiCaprio"}', 'name': 'wikipedia'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, id='run--784c39d8-8d90-492b-816f-22f3fde3862a', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'Leonardo DiCaprio'}, 'id': 'call_R0lWhR4lX7CfJZ8VgzavO47N', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'wikipedia', 'args': '{"query":"Leonardo DiCaprio"}', 'id': 'call_R0lWhR4lX7CfJZ8VgzavO47N', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_R0lWhR4lX7CfJZ8VgzavO47N'),
                    